# Milk Delivery for Hawker Centers in Singapore

Jack is a driver of ABC Milk Company. He is assigned to deliver fresh milk for 20 hawker centers each morning.  Suppose he can visit each center only once, could you help Jack to design a rounte with least amount of time (distance)?

The geolocation data of the hawker centers are imported as follows:

In [1]:
import pandas as pd
import numpy as np
import mpu
import pickle
import folium
import random
from folium.plugins import BeautifyIcon

import openrouteservice as ors
from rsome import ro
from rsome import grb_solver as grb

In [2]:
## Import the geolocation data of all the hawker centres
data = pd.read_csv('hawker_centres_kml.csv')
# print(data.head(5))

In [5]:
lon = data['X']
lat = data['Y']

N = 20
lon = [lon[i] for i in range(N)]
lat = [lat[i] for i in range(N)]


## plot the locations in singapore map
# https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/
SGmap = folium.Map(location=[lat[2], lon[2]], tiles="Stamen Terrain", zoom_start=11) #Stamen Terrain, OpenStreetMap
for i in range(N):
    folium.Marker(
        location=[lat[i], lon[i]],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(i + 1),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(SGmap)
SGmap

You searched online and realized that this exactly the traveling salesman problem, which can be solved by using the MTZ formulation. 

**(Miller–Tucker–Zemlin formulation)** Label the centres with the numbers $1,\ldots,N$ and define:
$$
x_{ij} = \begin{cases} 
1 & \text{the path goes from centre } i \text{ to centre } j \\ 
0 & \text{otherwise} 
\end{cases}
$$

For $i=1,\ldots,N$, let $u_i$ be a dummy variable, and finally take $d_{ij} > 0$ to be the distance from centre $i$ to centre $j$. Then TSP can be written as the following integer linear programming problem:
$$
\begin{align}
\min &\sum_{i=1}^n \sum_{j=1}^n d_{ij}x_{ij}\colon &&  \\
     & \sum_{i=1}^N x_{ij} = 1 && j=1, \ldots, N; \\
     & \sum_{j=1}^N x_{ij} = 1 && i=1, \ldots, N; \\
     & u_i-u_j +Nx_{ij} \le N-1 && 2 \le i \ne j \le N;  \\
     & 1 \le u_i \le N-1 && 2 \le i \le N; \\
     & u_{i} \in \mathbf{Z} && i=2, \ldots, N; \\
     & x_{ij} \in \{0,1\}  && i,j=1, \ldots, N; \\
     & x_{ii} = 0  && i= 1, \ldots, N; \\
\end{align}
$$

In [3]:
def Hawker_Centre_TSP_MTZ(N, distance):
    ## Miller–Tucker–Zemlin formulation
    m = ro.Model('Hawker Centre TSP-MTZ')
    x = m.dvar((N,N),'B')
    u = m.dvar(N, 'I')
    m.min( (distance * x).sum() )

    m.st( x[i,i] == 0 for i in range(N) )

    m.st( x.sum(axis = 0) == 1, 
         x.sum(axis = 1) == 1 )

    for i in range(1,N):
        m.st( u[i] <= N - 1, u[i] >= 1 )
        for j in range(1,N):
            m.st( u[i] - u[j] + N*x[i,j] <= N - 1 )

    m.solve(grb)
    x_star = x.get()
    obj = m.get()
    return x_star, obj

There are two parameters for this model: number of centers $N$ and distrance matrix $\boldsymbol{d}$. 

The number of centers $N$ is 20 for this problem, but the distance matrix is not known. However, you can approximately obtain it via the `haversine_distance` method of `mpu` package as follows:

In [8]:
distance = {}
distance_list = np.array([mpu.haversine_distance((lat[i], lon[i]), (lat[j], lon[j])) for i in range(N) for j in range(N)])
print(distance_list[0:10])
distance_mat = distance_list.reshape((N, -1))
print(distance_mat[0:5,0:10])

[ 0.         14.27513162 12.99972745  8.72153894 11.52260332  6.7988958
 11.97671042 24.14814479  1.65334921 16.06523113]
[[ 0.         14.27513162 12.99972745  8.72153894 11.52260332  6.7988958
  11.97671042 24.14814479  1.65334921 16.06523113]
 [14.27513162  0.          9.53147876 10.00186281  7.48591419  7.55293026
   2.81877753 11.97729186 15.46351029  2.74619101]
 [12.99972745  9.53147876  0.          4.31931491  2.43567386  8.32056417
  10.1474408  12.66489521 13.19343745  8.69127212]
 [ 8.72153894 10.00186281  4.31931491  0.          3.52726119  5.31352838
   9.37152173 16.45255935  8.87462432 10.34984602]
 [11.52260332  7.48591419  2.43567386  3.52726119  0.          6.10582954
   7.7883485  12.98605158 11.98138888  7.1850314 ]]


Now you can solve the model to obtain the solution:

In [8]:
x_MTZ, obj_MTZ = Hawker_Centre_TSP_MTZ(N, distance_mat[0:N,0:N])

Academic license - for non-commercial use only - expires 2022-07-30
Using license file C:\Users\kings\gurobi.lic
Being solved by Gurobi...
Solution status: 2
Running time: 990.6278s


It takes 990 seconds to solve the problem! Very time-consuming! So it is better to save the results. You can do so by using the `pickle` package:

In [9]:
results_MTZ = x_MTZ, obj_MTZ
## save the solution to pickle 
string = 'Hawker_Centre_TSP_MTZ.pickle'
with open(string, 'wb') as f:
    pickle.dump(results_MTZ, f)

So that next time you can directly load the result from pickle files without solving the model again:

In [11]:
x_tsp, obj_tsp = pickle.load( open( 'Hawker_Centre_TSP_MTZ.pickle', "rb" ) )
print(x_tsp)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.

In [12]:
np.argwhere( x_tsp )

array([[ 0,  8],
       [ 1, 16],
       [ 2,  7],
       [ 3, 18],
       [ 4,  2],
       [ 5,  0],
       [ 6,  5],
       [ 7, 11],
       [ 8,  3],
       [ 9, 14],
       [10,  6],
       [11,  9],
       [12,  1],
       [13, 15],
       [14, 13],
       [15, 12],
       [16, 10],
       [17,  4],
       [18, 19],
       [19, 17]], dtype=int64)

You can also obtain the route via the following function:

In [14]:
## obtain [lon, lat] pairs for the route
def obtain_route(x_star, lon, lat, K = 1):
    N = len(lon)
    seq_route = -np.ones((K,N + 1)) # route sequence
    lon_route = -np.ones((K,N + 1))
    lat_route = -np.ones((K,N + 1))

    # starting point
    lon_route[:,0] = lon[0]
    lat_route[:,0] = lat[0]
    seq_route[:,0]= 1

    ind = np.argwhere( x_star )

    for k in range(K):
        lon_route[k,1] = lon[ind[k,1]]
        lat_route[k,1] = lat[ind[k,1]]
        seq_route[k,1]= ind[k,1] + 1


        ind_j = ind[k][1]
        i = 1
        while ind_j != 0:
            i = i + 1
            seq_route[k,i] = ind[ind_j + K - 1,1] + 1
            lon_route[k,i] = lon[ind[ind_j + K - 1,1]]
            lat_route[k,i] = lat[ind[ind_j + K - 1,1]]

            ind_j = ind[ind_j + K - 1,1]
        
    return seq_route, lon_route, lat_route

In [15]:
seq_route_tsp, lon_route_tsp, lat_route_tsp = obtain_route(x_tsp, lon[0:N], lat[0:N])
print(seq_route_tsp[0])

[ 1.  9.  4. 19. 20. 18.  5.  3.  8. 12. 10. 15. 14. 16. 13.  2. 17. 11.
  7.  6.  1.]


Now you can plot the (approximate) routes on the map:

In [13]:
lon_route = lon_route_tsp[0]
lat_route = lat_route_tsp[0]
for i in range(N):
    folium.PolyLine([[lat_route[i], lon_route[i]], [lat_route[i + 1], lon_route[i + 1]]]).add_to(SGmap)
SGmap

### Some issues with the solution:

1. Distance is an approximation;

2. Routes are not on the real roads.

You can use `openrouteservice` to estimate the distance matrix and also use this package to draw the real routes for the example.

To use `openrouteservice`, you need to sign up in https://openrouteservice.org/dev/#/signup and request an API Key. 

In [4]:
## Import the geolocation data of all the hawker centres
data = pd.read_csv('hawker_centres_kml.csv')
# print(data.head(5))

## obtain the distance matrix
lon = data['X']
lat = data['Y']
N = 20 ## maximum locations = 50
lon = [lon[i] for i in range(N)]
lat = [lat[i] for i in range(N)]
latlon = [[lat[i], lon[i]] for i in range(N)]
lonlat = [[lon[i], lat[i]] for i in range(N)]

import requests
body = {"locations":lonlat[0:50],"metrics":["distance"]} ## maximum 50 locations

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62488317235a3dc24e77b257d920e034b0fc',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
print(call.status_code, call.reason)
data_req = call.json()
distance_mat = np.array(data_req['distances'])/1000

200 OK


In [15]:
## plot the selected locations in a new map
TSPmap1 = folium.Map(location=[lat[2], lon[2]], tiles="Stamen Terrain", zoom_start=11.5) #Stamen Terrain, OpenStreetMap
for i in range(N):
    folium.Marker(
        location=[lat[i], lon[i]],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(i + 1),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(TSPmap1)
TSPmap1

In [16]:
results = Hawker_Centre_TSP_MTZ(N, distance_mat)
## save the solution to pickle 
string = 'Hawker_Centre_TSP_MTZ_real.pickle'
with open(string, 'wb') as f:
    pickle.dump(results, f)

Being solved by Gurobi...
Solution status: 2
Running time: 42.3125s


This problem has the same size as previous one, but it only takes 42 seconds! It is because this is a integer programming problem. 

In [16]:
x_tsp, obj_tsp = pickle.load( open( 'Hawker_Centre_TSP_MTZ_real.pickle', "rb" ) )
seq_route_tsp, lon_route_tsp, lat_route_tsp = obtain_route(x_tsp, lon[0:N], lat[0:N])
print(seq_route_tsp)

[[ 1.  9.  4.  3.  5. 18. 20. 19.  8. 12. 10.  2. 15. 14. 13. 16. 17.  7.
  11.  6.  1.]]


In [18]:
lon_route = lon_route_tsp[0]
lat_route = lat_route_tsp[0]
for i in range(N):
    # Specify your personal API key, registered in openrouteservice.com
    client = ors.Client(key = '5b3ce3597851110001cf62488317235a3dc24e77b257d920e034b0fc') 
    coordinates = [[lon_route[i], lat_route[i]], [lon_route[i + 1], lat_route[i + 1]]]

    route = client.directions(coordinates = coordinates, profile = 'foot-walking', format = 'geojson') #driving-car, cycling-road, cycling-regular, foot-walking
    
    gj = folium.GeoJson(
        name='route',
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": route['features'][0]['geometry']}]},
    )
    gj.add_to(TSPmap1)

folium.LayerControl().add_to(TSPmap1)
TSPmap1

If you want to visit all the 125 hawker centers, you can load the solution from "Hawker_Centre_TSP_MTZ_All.pickle" and plot the routes as follows:

In [22]:
## Import the geolocation data of all the hawker centres
data = pd.read_csv('hawker_centres_kml.csv')
# print(data.head(5))

## obtain the distance matrix
lon = data['X'].values
lat = data['Y'].values
N = 125
latlon = [[lat[i], lon[i]] for i in range(N)]
lonlat = [[lon[i], lat[i]] for i in range(N)]

x_tsp, obj_tsp = pickle.load( open( 'Hawker_Centre_TSP_MTZ_All.pickle', "rb" ) )

seq_route_tsp, lon_route_tsp, lat_route_tsp = obtain_route(x_tsp, lon[0:N], lat[0:N])

In [23]:
lon_route = lon_route_tsp[0]
lat_route = lat_route_tsp[0]
SGmap1 = folium.Map(location=[lat[2], lon[2]], tiles="Stamen Terrain", zoom_start=11.4) #Stamen Terrain, OpenStreetMap
for i in range(N):
    folium.Marker(
        location=[lat[i], lon[i]],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(i + 1),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(SGmap1)
    
for i in range(N):
    client = ors.Client(key = '5b3ce3597851110001cf62488317235a3dc24e77b257d920e034b0fc') # Specify your personal API key, registered in openrouteservice.com

    coordinates = [[lon_route[i], lat_route[i]], [lon_route[i + 1], lat_route[i + 1]]]

    route = client.directions(coordinates = coordinates, profile = 'foot-walking', format = 'geojson') #driving-car, cycling-road, cycling-regular, foot-walking
    
    gj = folium.GeoJson(
        name='route',
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": route['features'][0]['geometry']}]},
    )
    gj.add_to(SGmap1)

folium.LayerControl().add_to(SGmap1)
SGmap1

C:\ProgramData\Anaconda3\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\ProgramData\Anaconda3\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\ProgramData\Anaconda3\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\ProgramData\Anaconda3\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\ProgramData\Anaconda3\lib\site-packages\openrouteservice\client.py:211: U

### Extend it to a vehicle routing problem (optional)

**Due to vehicle capacity constraints, Jack can only deliver milk to some of the centers each time. To put it another way, Jack must deliver at least two times. Could you assist Jack in planning the two routes for each day? Assume that Jack can only visit each center once.**

Let $\mathcal{V} = \{0, 1,2,\ldots, N\}$. The formulation of the TSP can be extended to create the two index vehicle flow formulations for the VRP

$$ 
\begin{align}
\min & \sum_{i\in \mathcal{V}}\sum_{j \in \mathcal{V}}c_{ij}x_{ij}\\
\mbox{s.t.} & \sum_{i \in \mathcal{V}}x_{ij}=1 \quad && \forall j \in \mathcal{V}\backslash \left \{ 0 \right \} \\
&\sum_{j \in \mathcal{V}}x_{ij}=1 \quad && \forall i \in \mathcal{V}\backslash \left \{ 0 \right \}\\
&\sum_{i \in \mathcal{V}}x_{i0}=K&& \\
&\sum_{j \in \mathcal{V}}x_{0j}=K&& \\
&u_j-u_i\geq d_j-C(1-x_{ij}) && \forall i,j \in V\backslash\{0\}, i\neq j~~~~\text{s.t. } d_i +d_j \leq C\\
&0 \leq u_i \leq C-d_i && \forall i \in V\backslash \{0\}\\
&x_{ij}\in \{0,1\} \quad && \forall i,j \in \mathcal{V}\\
&x_{ii} = 0 && \forall i \in \mathcal{V}
\end{align}
$$

In this formulation $c_{ij}$ represents the cost of going from node $i$ to node $j$, $x_{ij}$ is a binary variable that has value $1$ if the arc going from $i$ to $j$ is considered as part of the solution and $0$ otherwise, $K$ is the number of available vehicles. We are also assuming that $0$ is the depot node.

The first and second constraints state that exactly one arc enters and exactly one leaves each vertex associated with a customer, respectively. The third and fourth  say that the number of vehicles leaving the depot is the same as the number entering. The fifth and sixth constraints are the capacity cut constraints, which impose that the routes must be connected and that the demand on each route must not exceed the vehicle capacity. Finally, the sixth constraints are the integrality constraints and the last constraints ensure that no arc for each node.

The fifth and sixth constraints are known as the MTZ constraints, they were first proposed for the TSP and subsequently extended by Christofides, Mingozzi and Toth
$$
u_j-u_i\geq d_j-C(1-x_{ij}) ~~~~~~\forall i,j \in V\backslash\{0\}, i\neq j~~~~\text{s.t. } d_i +d_j \leq C
$$
and
$$
0 \leq u_i \leq C-d_i ~~~~~~\forall i \in V\backslash \{0\}
$$
where $ u_i,~i \in V \backslash \{0\} $ is an additional continuous variable which represents the load left in the vehicle '''after''' visiting customer $i$ and $d_i$ is the demand of customer $i$. These impose both the connectivity and the capacity requirements. When $x_{ij}=0$ constraint then $i$ is not binding' since $u_i\leq C$ and $u_j\geq d_j$ whereas $x_{ij} = 1$ they impose that $u_j \geq u_i +d_j$.


In [5]:
def Hawker_Centre_VRP_MTZ(N, distance, Capacity, K, demand):
    ## Miller–Tucker–Zemlin formulation
    m = ro.Model('Hawker Centre VRP-MTZ')
    x = m.dvar( (N, N),'B' )
    u = m.dvar( N )
    m.min( (distance * x).sum() )
    
    m.st( x[i,i] == 0 for i in range(N) )
    
    m.st( x[:,j].sum() == 1 for j in range(1,N) ) 
    m.st( x[i,:].sum() == 1 for i in range(1,N) )
    
    m.st( x[:,0].sum() == K, 
         x[0,:].sum() == K )

    for i in range(1,N):
        m.st( u[i] <= Capacity - demand[i], u[i] >= 0 )
        for j in range(1,N):
            if j != i and demand[i] + demand[j] <= Capacity:
                m.st( u[j] - u[i] >= demand[j] - Capacity*(1 - x[i,j]) )

    m.solve(grb)
    x_star = x.get()
    obj = m.get()
    return x_star, obj

In [6]:
random.seed(3)
N = 20
K = 2
Capacity = 400
demand = [10,20,15,25,30,5,50,10,40,20,15,10,10,20,30,10,10,10,20,30]
distance_mat = distance_mat.astype(int)

In [7]:
results = Hawker_Centre_VRP_MTZ(N, distance_mat, Capacity, K, demand)
## save the solution to pickle 
string = 'Hawker_Centre_VRP_MTZ_real.pickle'
with open(string, 'wb') as f:
    pickle.dump(results, f)

Academic license - for non-commercial use only - expires 2022-07-30
Using license file C:\Users\kings\gurobi.lic
Being solved by Gurobi...
Solution status: 2
Running time: 5.0714s


In [8]:
x_vrp, obj_vrp = pickle.load( open( 'Hawker_Centre_VRP_MTZ_real.pickle', "rb" ) )

seq_route_vrp, lon_route_vrp, lat_route_vrp = obtain_route(x_vrp, lon[0:N], lat[0:N],K)
seq_route_vrp

array([[ 1.,  6.,  7., 11., 17.,  2., 13., 16., 14., 15.,  8., 12., 10.,
         4.,  3.,  5., 18., 20., 19.,  1., -1.],
       [ 1.,  9.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1.]])

If you set a lower capacity or more vehicles are needed, this problem tends to be very time consuming! There are many other formulations that are better then the MTZ and there are many heuristics that can help solve the problem to (near) optimality, which will not be covered in this course. 